In [1]:
import pandas as pd
from triplet_loss import sigmoidL2
import json
import torch

In [2]:
df = pd.read_csv('encodings.csv',index_col=0)

In [3]:
df['encoding'] = [json.loads(i) for i in df['encoding'].values.tolist()]

In [4]:
attr = pd.read_csv('attributes.csv',index_col=0)

In [5]:
listOfDFS = []
for type in ['train','test','validation']:
    print('Type:', type)
    subAttr = attr[attr['type'] == type]
    
    subDF = pd.merge(df, subAttr[['path','type']])
    
    subDF['key'] = 0
    
    crossJoined = subDF.merge(subDF, on='key', how='outer')
    
    crossJoined = crossJoined[crossJoined['path_x'] != crossJoined['path_y']]
    
    leftEncoding = torch.tensor(crossJoined['encoding_x'].values.tolist())
    
    rightEncoding = torch.tensor(crossJoined['encoding_y'].values.tolist())
    
    distances = sigmoidL2(leftEncoding, rightEncoding)
    
    crossJoined['distance'] = distances
    
    closest = crossJoined[
    [
        'path_x', 
        'path_y',
        'distance'
    ]
    ].groupby(
        'path_x', 
        sort=False).apply(
        lambda x: x.nsmallest(5, 'distance')).reset_index(drop=True)
    
    closest = closest.groupby('path_x', as_index=False).agg(list).drop(['distance'],axis=1)
    
    closest = closest.rename({'path_x':'path', 'path_y':'closestPaths'}, axis=1)
    
    listOfDFS.append(closest)

Type: train
Type: test
Type: validation


In [10]:
attr = pd.merge(attr, pd.concat(listOfDFS))

In [11]:
attr.to_csv('attributes_with_closest.csv')